# TSA Chapter 7: Case Study - Cointegration Test

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_cointegration/TSA_ch7_case_cointegration.ipynb)

Engle-Granger cointegration test on FRED GS10 and TB3MS interest rates: OLS regression, residual analysis, and ACF of residuals.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# OLS: Short = alpha + beta * Long + residual
X = sm.add_constant(data['GS10'])
reg = OLS(data['TB3MS'], X).fit()
resid = reg.resid
adf_r = adfuller(resid)
acf_resid = acf(resid, nlags=25, fft=True)
ci = 1.96 / np.sqrt(len(resid))

print(f'OLS: TB3MS = {reg.params.iloc[0]:.3f} + {reg.params.iloc[1]:.3f} * GS10, R^2 = {reg.rsquared:.3f}')
print(f'ADF on residuals: t-stat={adf_r[0]:.3f}, p-value={adf_r[1]:.4f}')

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Panel 1: Scatter + OLS
axes[0].scatter(data['GS10'], data['TB3MS'], color=BLUE, alpha=0.4, s=10, label='Monthly obs.')
x_line = np.linspace(data['GS10'].min(), data['GS10'].max(), 100)
axes[0].plot(x_line, reg.params.iloc[0] + reg.params.iloc[1] * x_line, color=RED, lw=2,
             label=f'OLS ($R^2$={reg.rsquared:.3f})')
axes[0].set_xlabel('10-Year Rate (%)')
axes[0].set_ylabel('3-Month Rate (%)')
axes[0].set_title('Cointegrating Regression', fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

# Panel 2: Residuals over time
axes[1].plot(data.index, resid, color=GREEN, lw=0.8)
axes[1].axhline(0, color=GRAY, ls='--', lw=0.8)
col = GREEN if adf_r[1] < 0.05 else RED
axes[1].set_title(f'OLS Residuals (ADF p={adf_r[1]:.4f})', fontweight='bold', color=col)
axes[1].set_ylabel('Residual')

# Panel 3: ACF of residuals
axes[2].bar(range(len(acf_resid)), acf_resid, color=PURPLE, width=0.6)
axes[2].axhline(ci, color=RED, ls='--', lw=0.8)
axes[2].axhline(-ci, color=RED, ls='--', lw=0.8)
axes[2].axhline(0, color='black', lw=0.5)
axes[2].set_title('ACF of Residuals', fontweight='bold')
axes[2].set_xlabel('Lag')
axes[2].set_ylabel('ACF')

plt.tight_layout()
save_chart(fig, 'ch7_case_cointegration')
plt.show()